# Choke Position Model Verification
Notebook to load a Parquet evaluation dataset, run ONNX model predictions, compute residual metrics / anomaly flags, visualize, and export an evaluation report.

Model type here: XGBoost residual or Isolation Forest? We focus first on residual_downP style then choke_position IF. Adjust target/paths as needed.

## 1. Load Dependencies and Configuration

In [1]:
# Install runtime deps if missing (idempotent)
import importlib, sys, subprocess
for pkg in ['onnxruntime','seaborn','scipy']:
    if importlib.util.find_spec(pkg) is None:
        subprocess.check_call([sys.executable,'-m','pip','install', pkg])

from pathlib import Path
import pandas as pd, numpy as np, json, math, time
import onnxruntime as ort
import seaborn as sns, matplotlib.pyplot as plt
from scipy import stats

# --- Configurable paths ---
parquet_path   = Path('training_data/synth_choke_position.parquet')  # change to real eval parquet as needed
models_dir     = Path('models_3')                                    # directory containing ONNX models
model_filename = 'choke_position.onnx'                               # model under test (IsolationForest)
mad_json_path  = models_dir/'residual_mad.json'                      # only used for residual models
target_column  = 'Choke-Position'                                    # true value column (for residual models)
timestamp_col  = None                                                # set to column name if time axis present
results_dir    = Path('summary') / 'model_eval'
results_dir.mkdir(parents=True, exist_ok=True)
print('Configured model path:', models_dir / model_filename)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 15.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [onnxruntime] [onnxruntime]
Configured model path: models_3/choke_position.onnx


## 2. Load Parquet Evaluation Dataset

In [3]:
df_eval = pd.read_parquet(parquet_path)
print('Shape:', df_eval.shape)
display(df_eval)

Shape: (10, 4)


,Choke-Position,ToolStateNum,Downstream-Temperature,is_anomaly
0,77.395605,2,16.925677,0
1,69.736803,1,13.846731,0
2,76.113970,5,15.774798,0
3,45.038594,2,16.966688,0
4,64.386512,6,16.501264,0
5,22.723872,2,14.361676,0
6,-1.800000,2,15.500000,1
7,102.500000,2,16.000000,1
8,50.000000,7680,16.200000,1
9,60.000000,2,105.000000,1


## 3. Quick Data Integrity & Summary Checks

In [5]:
print('dtypes:')
print(df_eval.dtypes)
print('Null counts:')
print(df_eval.isna().sum())
display(df_eval.describe(include='all'))
if target_column in df_eval:
    print('Target column present.')
else:
    print('Warning: target column not present; residual metrics limited.')
# Ensure no all-null feature columns
all_null_cols = [c for c in df_eval.columns if df_eval[c].isna().all()]
assert not all_null_cols, f'All-null columns detected: {all_null_cols}'

dtypes:
Choke-Position            float64
ToolStateNum                int64
Downstream-Temperature    float64
is_anomaly                  int64
dtype: object
Null counts:
Choke-Position            0
ToolStateNum              0
Downstream-Temperature    0
is_anomaly                0
dtype: int64


,Choke-Position,ToolStateNum,Downstream-Temperature,is_anomaly
count,10.000000,10.000000,10.000000,10.000000
mean,56.609536,770.400000,24.707683,0.400000
std,29.627699,2427.786472,28.230139,0.516398
min,-1.800000,1.000000,13.846731,0.000000
25%,46.278945,2.000000,15.568700,0.000000
50%,62.193256,2.000000,16.100000,0.000000
75%,74.519678,4.250000,16.819574,1.000000
max,102.500000,7680.000000,105.000000,1.000000


Target column present.


## 4. Load ONNX Model & Extract Feature Order

In [6]:
sess = ort.InferenceSession((models_dir/model_filename).as_posix(), providers=['CPUExecutionProvider'])
meta = sess.get_modelmeta().custom_metadata_map
feat_order = meta.get('feature_names','').split(',') if 'feature_names' in meta else [i.name for i in sess.get_inputs()]
feat_order = [f for f in feat_order if f]  # strip empties
print('Feature order from model:', feat_order)
print('Input shape expectation (N,', len(feat_order),')')

Feature order from model: ['Choke-Position', 'ToolStateNum', 'Downstream-Temperature']
Input shape expectation (N, 3 )


## 5. Define Log Scaling & Feature Engineering Helpers

In [7]:
LOG_FEATURES = ['Upstream-Pressure','Downstream-Pressure','Downstream-Upstream-Difference']
def make_X_eval(df, cols, target=None):
    X = df[cols].copy()
    for c in LOG_FEATURES:
        if c in X.columns and c != target:
            X[c] = np.log1p(X[c].clip(lower=0))
    return X.astype('float32').values

def compute_metrics(y_true, y_pred):
    resid = y_true - y_pred
    abs_resid = np.abs(resid)
    mae = float(abs_resid.mean())
    mse = float((resid**2).mean())
    rmse = math.sqrt(mse)
    # R2 manual
    ss_res = float((resid**2).sum())
    ss_tot = float(((y_true - y_true.mean())**2).sum())
    r2 = 1 - ss_res/ss_tot if ss_tot else float('nan')
    return {'MAE':mae,'RMSE':rmse,'R2':r2,'MaxAbsResid': float(abs_resid.max())}

## 6. Build Feature Matrix Matching Training Pipeline

In [8]:
X_eval = make_X_eval(df_eval, feat_order, target=None)  # IsolationForest input (no target exclusion)
print('X_eval shape:', X_eval.shape)
# For IF, y_true is not used; for residual model you would set y_true below.
y_true = df_eval[target_column].astype('float32').values if target_column in df_eval else None

X_eval shape: (10, 3)


## 7. Run Batch Prediction

In [9]:
outputs = sess.run(None, {'input': X_eval})
# IsolationForest ONNX typically returns labels then scores
if len(outputs)==2:
    labels, scores = outputs
    labels = labels.squeeze()
    scores = scores.squeeze()
    print('Labels sample:', labels[:10])
    print('Scores sample:', np.round(scores[:10],4))
else:
    preds = outputs[0].squeeze()
    print('Pred sample:', preds[:10])

Labels sample: [ 1  1  1  1  1  1  1  1 -1 -1]
Scores sample: [ 0.1381  0.0719  0.0558  0.1115  0.0829  0.0952  0.1035  0.2956 -0.0304
 -0.0187]


## 8. Compute Residuals & Core Metrics (MAE / RMSE / R2)

In [10]:
is_residual = False  # set True if evaluating residual regression model instead of IF
if is_residual and y_true is not None:
    y_pred = outputs[0].squeeze().astype('float32')
    metrics = compute_metrics(y_true, y_pred)
    display(metrics)
else:
    print('Skipping residual metrics because model is IsolationForest (unsupervised).')

Skipping residual metrics because model is IsolationForest (unsupervised).


## 9. Residual Percentiles & MAD Based Cutoff Comparison

In [11]:
cutoff = None
mad_limits = None
if mad_json_path.exists():
    try:
        mad_limits = json.loads(mad_json_path.read_text())
    except Exception as e:
        print('Failed to load MAD JSON:', e)
if is_residual and mad_limits and target_column in mad_limits:
    cutoff = mad_limits[target_column]['cutoff']
    resid = (y_true - y_pred)
    abs_resid = np.abs(resid)
    pct = np.percentile(abs_resid,[50,90,95,97.5,99])
    print('Cutoff:', cutoff)
    print('Percentiles 50/90/95/97.5/99:', pct)
else:
    print('No residual percentile analysis (IF model or missing MAD JSON).')

No residual percentile analysis (IF model or missing MAD JSON).


## 10. Flag Anomalies & Aggregate Alert Statistics

In [12]:
if len(outputs)==2:  # IsolationForest path
    anomaly_mask = (labels == -1)
    rate = anomaly_mask.mean()*100
    print(f'Anomaly rate: {rate:.2f}% ({anomaly_mask.sum()}/{len(anomaly_mask)})')
    if 'is_anomaly' in df_eval:
        tp = (anomaly_mask & (df_eval['is_anomaly']==1)).sum()
        fp = (anomaly_mask & (df_eval['is_anomaly']==0)).sum()
        fn = ((~anomaly_mask) & (df_eval['is_anomaly']==1)).sum()
        prec = tp/(tp+fp) if (tp+fp)>0 else float('nan')
        rec  = tp/(tp+fn) if (tp+fn)>0 else float('nan')
        print(f'TP={tp} FP={fp} FN={fn} Precision={prec:.2f} Recall={rec:.2f}')
else:
    print('Not an IF output; define anomaly rule for regression if needed.')

Anomaly rate: 20.00% (2/10)
TP=2 FP=0 FN=2 Precision=1.00 Recall=0.50


## 11. Actual vs Predicted Scatter Plot

In [13]:
if is_residual and y_true is not None:
    plt.figure(figsize=(5,5))
    plt.scatter(y_true, y_pred, s=12, alpha=0.6)
    lims=[min(y_true.min(), y_pred.min()), max(y_true.max(), y_pred.max())]
    plt.plot(lims, lims, 'r--', label='y=x')
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    plt.title('Actual vs Predicted')
    plt.legend()
    plt.show()
else:
    print('Scatter skipped (IF model).')

Scatter skipped (IF model).


## 12. Residual Histogram & QQ Plot

In [14]:
if is_residual and y_true is not None:
    resid = y_true - y_pred
    fig, axes = plt.subplots(1,3, figsize=(14,4))
    sns.histplot(resid, kde=True, ax=axes[0]); axes[0].set_title('Residuals')
    sns.histplot(np.abs(resid), kde=True, ax=axes[1]); axes[1].set_title('|Residuals|')
    stats.probplot(resid, dist='norm', plot=axes[2])
    axes[2].set_title('QQ Plot')
    plt.tight_layout(); plt.show()
else:
    print('Residual plots skipped (IF model).')

Residual plots skipped (IF model).


## 13. Time Series Plot With Anomaly Overlays

In [15]:
if timestamp_col and timestamp_col in df_eval and len(outputs)==2:
    df_ts = df_eval.copy().sort_values(timestamp_col)
    df_ts['anomaly'] = (labels==-1)
    plt.figure(figsize=(10,4))
    plt.plot(df_ts[timestamp_col], df_ts[target_column] if target_column in df_ts else df_ts[feat_order[0]], label='Signal')
    plt.scatter(df_ts.loc[df_ts.anomaly, timestamp_col],
                (df_ts.loc[df_ts.anomaly, target_column] if target_column in df_ts else df_ts.loc[df_ts.anomaly, feat_order[0]]),
                color='red', s=30, label='Anomaly')
    plt.legend(); plt.title('Time Series with Anomalies'); plt.show()
else:
    print('Time series plot skipped (missing timestamp or not IF).')

Time series plot skipped (missing timestamp or not IF).


## 14. Per-Window Stability / Drift Checks (Optional)

In [16]:
if timestamp_col and timestamp_col in df_eval and is_residual and y_true is not None:
    df_eval['_resid'] = y_true - y_pred
    win = '1H'  # adjust resample window
    drift = df_eval.set_index(timestamp_col)['_resid'].resample(win).agg(['mean','std'])
    display(drift.head())
    drift.plot(subplots=True, figsize=(10,4), title='Residual Drift'); plt.tight_layout()
else:
    print('Drift check skipped (need timestamp + residual model).')

Drift check skipped (need timestamp + residual model).


## 15. Evaluate Multiple Models in Directory (Loop)

In [17]:
def evaluate_if_model(fp, df):
    s = ort.InferenceSession(fp.as_posix(), providers=['CPUExecutionProvider'])
    feats = s.get_modelmeta().custom_metadata_map.get('feature_names','').split(',')
    feats = [f for f in feats if f] or [i.name for i in s.get_inputs()]
    X = make_X_eval(df, feats)
    lbl, *_ = s.run(None, {'input': X})
    lbl = lbl.squeeze()
    rate = (lbl==-1).mean()*100
    return {'file': fp.name, 'anomaly_rate_pct': rate, 'n': len(lbl)}

multi_summary = []
for fp in sorted(models_dir.glob('*.onnx')):
    if 'residual' in fp.name:  # skip residual for simplicity here
        continue
    try:
        multi_summary.append(evaluate_if_model(fp, df_eval))
    except Exception as e:
        print('Failed for', fp.name, e)
multi_df = pd.DataFrame(multi_summary)
display(multi_df)

Failed for delta_temp_open.onnx "None of [Index(['DeltaTemperature'], dtype='object')] are in the [columns]"
Failed for full_vectors_if.onnx "['Battery-Voltage', 'Upstream-Pressure', 'Downstream-Pressure', 'Downstream-Upstream-Difference', 'Upstream-Temperature'] not in index"
Failed for pressure_pair_open.onnx "None of [Index(['Upstream-Pressure', 'Downstream-Pressure'], dtype='object')] are in the [columns]"


,file,anomaly_rate_pct,n
0,choke_position.onnx,20.0,10


## 16. Export Evaluation Report & Artifacts

In [ ]:
report = {}
if len(outputs)==2:
    report['anomaly_rate_pct'] = float((labels==-1).mean()*100)
    if 'is_anomaly' in df_eval:
        anomaly_mask = labels==-1
        report['tp'] = int((anomaly_mask & (df_eval['is_anomaly']==1)).sum())
        report['fp'] = int((anomaly_mask & (df_eval['is_anomaly']==0)).sum())
        report['fn'] = int(((~anomaly_mask) & (df_eval['is_anomaly']==1)).sum())
        report['precision'] = report['tp']/ (report['tp']+report['fp']) if (report['tp']+report['fp'])>0 else None
        report['recall'] = report['tp']/ (report['tp']+report['fn']) if (report['tp']+report['fn'])>0 else None
else:
    if is_residual and y_true is not None:
        report.update(metrics)
(results_dir/'choke_position_eval.json').write_text(json.dumps(report, indent=2))
print('Saved report to', results_dir/'choke_position_eval.json')
# Save per-row if anomalies present
if len(outputs)==2:
    out_df = df_eval.copy()
    out_df['if_label'] = labels
    out_df.to_parquet(results_dir/'choke_position_eval_rows.parquet', index=False)
    print('Saved per-row parquet.')

## 17. Reusable Utility Functions Cell
(Copied earlier; consolidate if refactoring into module.)